# Cyoda Client Demo

This notebook demonstrates how to connect and interact with the Cyoda API.


## Prerequisites

Ensure you have the following before running the cells:

- Cyoda API credentials (API key, secret, etc.)

- Python packages installed via `requirements.txt`


### Steps

1. **Setup**: Import necessary libraries and set up environment variables.

2. **Authentication**: Authenticate with the Cyoda API.

3. **Operations**: Perform basic and advanced operations.


### Step 1: Setup Environment and Import Libraries
Ensure you have set up your environment properly.

In [ ]:

import logging

def setup_logging(level=logging.INFO):
    logging.basicConfig(level=level)
    logger = logging.getLogger(__name__)
    return logger

logger = setup_logging()
logger.info("Logging initialized.")


In [ ]:
import os

API_KEY = os.environ["CYODA_API_KEY"]
API_SECRET = os.environ["CYODA_API_SECRET"]
API_URL = os.environ["CYODA_API_URL"]+"/api"
GRPC_ADDRESS = os.environ["GRPC_ADDRESS"]
WORK_DIR = os.environ["WORK_DIR"]
TOKEN = ""
logger.info(f"API URL: {API_URL}")
logger.info(f"GRPC Address: {GRPC_ADDRESS}")


In [ ]:

# Define entity and model version constants
ENTITY_CLASS_NAME = "com.cyoda.tdb.model.treenode.TreeNodeEntity"
ENTITY_NAME = "prizes"
MODEL_VERSION = "1000"

logger.info(f"Using entity: {ENTITY_CLASS_NAME}, model version: {MODEL_VERSION}")


In [ ]:
# Let's do the auth first
import requests
import json

def authenticate(api_key, api_secret, api_url):
    login_url = f"{api_url}/auth/login"
    headers = {"Content-Type": "application/json", "X-Requested-With": "XMLHttpRequest"}
    auth_data = {"username": api_key, "password": api_secret}

    logger.info("Attempting to authenticate with Cyoda API.")
    
    try:
        response = requests.post(login_url, headers=headers, data=json.dumps(auth_data), timeout=10)
        
        if response.status_code == 200:
            token = response.json().get("token")
            logger.info("Authentication successful!")
            return token
        else:
            logger.error(f"Authentication failed with {response}")
            return None
    
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        return None

TOKEN = authenticate(API_KEY, API_SECRET, API_URL)

### Let's define several supplementary functions

In [ ]:
def send_get_request(path):
    url = f"{API_URL}/{path}"

    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {TOKEN}"}
    response = requests.get(url, headers=headers)
    return response

In [ ]:
def send_post_request(path, data):
    url = f"{API_URL}/{path}"

    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {TOKEN}"}
    response = requests.post(url, headers=headers, data=data)
    return response

In [ ]:
def send_put_request(path, data, timeout):
    url = f"{API_URL}/{path}"
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {TOKEN}"}
    response = requests.put(url, headers=headers, data=data, timeout=timeout)
    return response

In [ ]:
def send_delete_request(path):
    url = f"{API_URL}/{path}"
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {TOKEN}"}
    response = requests.delete(url, headers=headers)
    return response

In [ ]:

def read_file(file_path: str) -> str:
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except Exception as e:
        logger.error(f"Failed to read file at {file_path}: {e}")
        raise

## Cleaning up the env
Let's remove the data for the existing schema and the schema itself so we can start from scratch.

In [ ]:
def delete_entity_data(entity_name, version):
    path = f"entity/TREE/{entity_name}/{version}"
    try:
        response = send_delete_request(path=path)
        
        if response.status_code == 200:
            logger.info(f"Successfully deleted entity '{entity_name}' with version '{version}'. Response: {response}")
        else:
            logger.error(f"Failed to delete entity '{entity_name}' with version '{version}'. Response: {response}")
        
        return response
    
    except Exception as e:
        logger.error(f"An error occurred while deleting entity '{entity_name}' with version '{version}': {e}")
        return {'error': str(e)}

In [ ]:
response = delete_entity_data(ENTITY_NAME, MODEL_VERSION)
logger.info(f"Delete response: {response}")

In [ ]:
def delete_entity_schema(entity_name, version):
    try:
        path = f"treeNode/model/{entity_name}/{version}"
        response = send_delete_request(path=path)
        
        if response.status_code == 200:
            logger.info(f"Successfully deleted schema for entity '{entity_name}' with version '{version}'. Response: {response}")
        else:
            logger.error(f"Failed to delete schema for entity '{entity_name}' with version '{version}'. Status Code: {response}, Response: {response}")
        
        return response
    
    except Exception as e:
        logger.error(f"An error occurred while deleting schema for entity '{entity_name}' with version '{version}': {e}")
        return {'error': str(e)}

In [ ]:
response = delete_entity_schema(ENTITY_NAME, MODEL_VERSION)
logger.info(f"Delete response: {response}")

### Saving data
To begin, we'll first create the schema. Once the schema is in place, we will proceed to lock it to ensure its integrity. After that, we can move on to ingesting the data from the file into the system.

In [ ]:
def save_entity_schema(entity_name, version, data):
    path = f"treeNode/model/import/JSON/SAMPLE_DATA/{entity_name}/{version}"
    
    try:
        response = send_post_request(path=path, data=data)
        if response.status_code == 200:
            logger.info(f"Successfully saved schema for entity '{entity_name}' with version '{version}'. Response: {response}")
        else:
            logger.error(f"Failed to save schema for entity '{entity_name}' with version '{version}'. Response: {response}")
        
        return response
    
    except Exception as e:
        logger.error(f"An error occurred while saving schema for entity '{entity_name}' with version '{version}': {e}")
        return {'error': str(e)}

In [ ]:
def test_save_schema(entity_name, file_path):
    data = read_file(file_path)
    response = save_entity_schema(
        entity_name=entity_name, version=MODEL_VERSION, data=data
    )
    logger.info(f"Response: {response}")

file_path_base = f"{WORK_DIR}/example/prizes_schema.json"
test_save_schema(ENTITY_NAME, file_path_base)

In [ ]:
def lock_entity_schema(entity_name, version, data):
    path = f"treeNode/model/{entity_name}/{version}/lock"

    try:
        response = send_put_request(path=path, data=data, timeout=30)
        
        if response.status_code == 200:
            logger.info(f"Successfully locked schema for entity '{entity_name}' with version '{version}'. Response: {response}")
        else:
            logger.error(f"Failed to lock schema for entity '{entity_name}' with version '{version}'. Response: {response}")
        
        return response
    except Exception as e:
        logger.error(f"An error occurred while locking schema for entity '{entity_name}' with version '{version}': {e}")
        return {'error': str(e)}
    

In [ ]:
def test_lock_schema(entity_name):
    try:
        response = lock_entity_schema(entity_name=entity_name, version=MODEL_VERSION, data={})
        logger.info(f"Response: {response}")    
    except Exception as e:
        logger.error(f"An error occurred while testing schema locking for entity '{entity_name}': {e}")

test_lock_schema(ENTITY_NAME)

In [ ]:
def save_new_entity(entity_name, version, data):
    path = f"entity/JSON/TREE/{entity_name}/{version}"
    logger.info(f"Saving new entity to path: {path}")
    
    try:
        response = send_post_request(path=path, data=data)
        
        if response.status_code == 200:
            logger.info(f"Successfully saved new entity. Response: {response}")
        else:
            logger.error(f"Failed to save new entity. Response: {response}")
        
        return response
    
    except Exception as e:
        logger.error(f"An error occurred while saving new entity '{entity_name}' with version '{version}': {e}")
        return {'error': str(e)}

In [ ]:
def test_save_new_entity(entity_name, file_path):
    
    try:
        data = read_file(file_path)
        response = save_new_entity(entity_name=entity_name, version=MODEL_VERSION, data=data) 
        return response
    
    except Exception as e:
        logger.error(f"An error occurred while testing save_new_entity for '{entity_name}': {e}")
        raise

file_path_base = f"{WORK_DIR}/example/prizes_entities.json"
response = test_save_new_entity(ENTITY_NAME, file_path_base)


In [ ]:
response_json = response.json()
if 'entityIds' in response_json[0]:
    entity_id = response_json[0]['entityIds'][0]
    logger.info(f"Extracted entity ID: {entity_id}")
else:
    logger.error("Response JSON does not contain 'entityIds'.")

In [ ]:
def get_entity_current_state(entity_id):
    
    path = f"platform-api/entity-info/fetch/lazy?entityClass={ENTITY_CLASS_NAME}&entityId={entity_id}&columnPath=state"
    response = send_get_request(path=path)
    logger.info(response.json())
    return response
get_entity_current_state(entity_id)

In [ ]:
def get_entities(model, version):
    
    path = f"entity/TREE/{model}/{version}"
    response = send_get_request(path=path)
    logger.info(response.json())
    return response
get_entities(ENTITY_NAME, MODEL_VERSION)


### GPPC client
We are about to establish a gRPC bi-directional streaming connection. Initially, we will send a 'Join' event and expect to receive a 'Greet' event in response.

Next, we will save the 'Prizes' entity. This entity will transition from a 'None' state to a 'Notified' state, which will trigger an external processor to send us a calculation request. This request will then be printed out. 

#### Plese import example/export_workflow_for_TreeNodeEntity_prizes.json workflow before you proceed :)

In [ ]:
# Step 1: Install gRPC and tools
!pip install grpcio grpcio-tools

# Step 2: Compile proto files
!python -m grpc_tools.protoc -I. --python_out=. --pyi_out=. --grpc_python_out=. cyoda-cloud-api.proto

!python -m grpc_tools.protoc -I. --python_out=. --pyi_out=. --grpc_python_out=. cloudevents.proto

In [ ]:

from enum import Enum

class CloudEventType(str, Enum):
    BASE_EVENT = "BaseEvent"
    CALCULATION_MEMBER_JOIN_EVENT = "CalculationMemberJoinEvent"
    CALCULATION_MEMBER_GREET_EVENT = "CalculationMemberGreetEvent"
    ENTITY_PROCESSOR_CALCULATION_REQUEST = "EntityProcessorCalculationRequest"
    ENTITY_PROCESSOR_CALCULATION_RESPONSE = "EntityProcessorCalculationResponse"   

In [ ]:
import grpc
import uuid
import json
import asyncio
import cloudevents_pb2 as cloudevents_pb2
import cloudevents_pb2_grpc as cloudevents_pb2_grpc
import cyoda_cloud_api_pb2 as cyoda_cloud_api_pb2
import cyoda_cloud_api_pb2_grpc as cyoda_cloud_api_pb2_grpc
from cloudevents_pb2 import CloudEvent

TAGS = ["prizes"]
OWNER = "PLAY"

def create_cloud_event(event_id, source, event_type, data) -> CloudEvent:
    """Create a CloudEvent instance with the given parameters."""
    return CloudEvent(
        id=event_id,
        source=source,
        spec_version="1.0",
        type=event_type,
        text_data=json.dumps(data)
    )

def create_notification_event(data) -> CloudEvent:
    """Create a CloudEvent for a notification response."""
    return create_cloud_event(
        event_id=str(uuid.uuid4()),
        source="SimpleSample",
        event_type="EntityProcessorCalculationResponse",
        data={
            "requestId": data['requestId'],
            "entityId": data['entityId'],
            "owner": OWNER,
            "payload": data['payload'],
            "success": True
        }
    )

async def produce_events(queue):
    """Produce events and put them in the queue."""
    join_event = create_cloud_event(
        event_id=str(uuid.uuid4()),
        source="SimpleSample",
        event_type="CalculationMemberJoinEvent",
        data={"owner": OWNER, "tags": TAGS}
    )

    await queue.put(join_event)
    await asyncio.sleep(10)
    
    file_path = f"{WORK_DIR}/example/prizes_entities.json"
    test_save_new_entity(ENTITY_NAME, file_path)
    
    await asyncio.sleep(15)
    await queue.put(None)  # Signal completion
    raise asyncio.TimeoutError("Operation timed out!")

async def consume_events(queue):
    """Consume events from the queue and handle responses."""
    async with grpc.aio.secure_channel(GRPC_ADDRESS, grpc.ssl_channel_credentials()) as channel:
        stub = cyoda_cloud_api_pb2_grpc.CloudEventsServiceStub(channel)

        async def event_generator():
            while True:
                event = await queue.get()
                if event is None:
                    break
                yield event
                queue.task_done()

        async for response in stub.startStreaming(event_generator()):
            logger.info(f"Received response: {response}")
            data = json.loads(response.text_data)
            if data.get('processorName') == 'notify':
                logger.info(f"Processing notification data: {data}")
                notification_event = create_notification_event(data)
                await queue.put(notification_event)

async def main():
    """Main function to run producer and consumer tasks."""
    queue = asyncio.Queue()
    producer_task = asyncio.create_task(produce_events(queue))
    consumer_task = asyncio.create_task(consume_events(queue))

    await asyncio.gather(producer_task, consumer_task)

# Run the main function
await main()